In [1]:
import s3fs
import zarr

url = "s3://hrrrzarr/sfc/20210601/20210601_00z_anl.zarr/surface/TMP/surface/TMP"
fs = s3fs.S3FileSystem(anon=True)
temperature = zarr.open(s3fs.S3Map(url, s3=fs))
temperature

<zarr.core.Array (1059, 1799) float16>

In [2]:
new_store = zarr.DirectoryStore("data/example_temp")
temp = zarr.group(new_store)
temp["TMP"] = temperature[...]

In [3]:
temp.attrs["_CRS"] = "test"

In [4]:
! cat data/example_temp/.zattrs 

{
    "_CRS": "test"
}

Okay, so I can create the local data structure and modify the attributes. Now, I need to try to figure out a good representation of the CRS... which means I'll need to set up reading it in rio_tiler so I can know whether I succeeded or not.

In [5]:
import os
import pyproj
os.environ["PROJ_LIB"] = "/opt/anaconda3/envs/titiler/share/proj"
pyproj.datadir.set_data_dir("/opt/anaconda3/envs/titiler/share/proj")

/opt/anaconda3/envs/titiler/lib/python3.8/site-packages/pyproj/__init__.py:90: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [6]:
from rio_tiler.io import COGReader


with COGReader("data/example_temp") as image:
    print(image.dataset)  # rasterio opened dataset
    img = image.read()  
    print(f"Image crs: {image.crs}")

/opt/anaconda3/envs/titiler/lib/python3.8/site-packages/rasterio/__init__.py:277: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/opt/anaconda3/envs/titiler/lib/python3.8/site-packages/rio_tiler/io/cogeo.py:149: NoOverviewWarning: The dataset has no Overviews. rio-tiler performances might be impacted.
  warnings.warn(
ERROR 1: Unable to compute a transformation between pixel/line and georeferenced coordinates for data/example_temp. There is no affine transformation and no GCPs. Specify transformation option SRC_METHOD=NO_GEOTRANSFORM to bypass this check.


<open DatasetReader name='data/example_temp' mode='r'>


CPLE_AppDefinedError: Unable to compute a transformation between pixel/line and georeferenced coordinates for data/example_temp. There is no affine transformation and no GCPs. Specify transformation option SRC_METHOD=NO_GEOTRANSFORM to bypass this check.

In [7]:
proj_params = {'a': 6371229,
   'b': 6371229,
   'proj': 'lcc',
   'lon_0': 262.5,
   'lat_0': 38.5,
   'lat_1': 38.5,
   'lat_2': 38.5}

# This doesn't look like the right "projjson" format per the example in gdal documentation, but it's worth a try
temp.attrs["_CRS"] = {"projjson": proj_params}

In [8]:
with COGReader("data/example_temp") as image:
    print(image.dataset)  # rasterio opened dataset
    img = image.read()  
    print(f"Image crs: {image.crs}")

<open DatasetReader name='data/example_temp' mode='r'>


ERROR 1: Unable to compute a transformation between pixel/line and georeferenced coordinates for data/example_temp. There is no affine transformation and no GCPs. Specify transformation option SRC_METHOD=NO_GEOTRANSFORM to bypass this check.


CPLE_AppDefinedError: Unable to compute a transformation between pixel/line and georeferenced coordinates for data/example_temp. There is no affine transformation and no GCPs. Specify transformation option SRC_METHOD=NO_GEOTRANSFORM to bypass this check.

I don't know that it's actually a bad CRS vs some other bug in the library (i.e. we definitely aren't going to have GCPs even if we define a CRS as far as I know), but I can at least try converting the CRS to say the WKT format and seeing if that changes anything.

In [9]:
import cartopy.crs as ccrs

projection = ccrs.LambertConformal(central_longitude=262.5, 
                                   central_latitude=38.5, 
                                   standard_parallels=(38.5, 38.5),
                                    globe=ccrs.Globe(semimajor_axis=6371229,
                                                     semiminor_axis=6371229))

In [10]:
wkt = projection.to_wkt()
wkt

'PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["unknown",ELLIPSOID["unknown",6371229,0,LENGTHUNIT["metre",1,ID["EPSG",9001]]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8901]]],CONVERSION["unknown",METHOD["Lambert Conic Conformal (2SP)",ID["EPSG",9802]],PARAMETER["Latitude of false origin",38.5,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8821]],PARAMETER["Longitude of false origin",262.5,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8822]],PARAMETER["Latitude of 1st standard parallel",38.5,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Latitude of 2nd standard parallel",38.5,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8824]],PARAMETER["Easting at false origin",0,LENGTHUNIT["metre",1],ID["EPSG",8826]],PARAMETER["Northing at false origin",0,LENGTHUNIT["metre",1],ID["EPSG",8827]]],CS[Cartesian,2],AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["(N)",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]'

In [17]:
projection.to_proj4()

/opt/anaconda3/envs/titiler/lib/python3.8/site-packages/pyproj/crs/crs.py:1286: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


'+proj=lcc +lat_0=38.5 +lon_0=262.5 +lat_1=38.5 +lat_2=38.5 +x_0=0 +y_0=0 +R=6371229 +units=m +no_defs +type=crs'

In [11]:
temp.attrs["_CRS"] = {"wkt": wkt}

In [12]:
with COGReader("data/example_temp") as image:
    print(image.dataset)  # rasterio opened dataset
    img = image.read()  
    print(f"Image crs: {image.crs}")

<open DatasetReader name='data/example_temp' mode='r'>


ERROR 1: Unable to compute a transformation between pixel/line and georeferenced coordinates for data/example_temp. There is no affine transformation and no GCPs. Specify transformation option SRC_METHOD=NO_GEOTRANSFORM to bypass this check.


CPLE_AppDefinedError: Unable to compute a transformation between pixel/line and georeferenced coordinates for data/example_temp. There is no affine transformation and no GCPs. Specify transformation option SRC_METHOD=NO_GEOTRANSFORM to bypass this check.

Okay, so I'm still getting the same error, but the CRS is probably as good as it can be for this dataset. I'm going to check to see if gdal can at least read this zarr (with its CRS), and, if not, if another dataset with a more standard CRS (such as an EPSG) is readable.

In [15]:
! /opt/anaconda3/envs/titiler/bin/gdal_translate 'ZARR:"data/example_temp"' out.tif

Input file size is 1799, 1059
0...10...20...30...40...50...60...70...80...90...100 - done.


In [16]:
! /opt/anaconda3/envs/titiler/bin/gdalmdiminfo data/example_temp

{
  "type": "group",
  "driver": "Zarr",
  "name": "/",
  "attributes": {
    "_CRS": {
      "wkt": "PROJCRS[\"unknown\",BASEGEOGCRS[\"unknown\",DATUM[\"unknown\",ELLIPSOID[\"unknown\",6371229,0,LENGTHUNIT[\"metre\",1,ID[\"EPSG\",9001]]]],PRIMEM[\"Greenwich\",0,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8901]]],CONVERSION[\"unknown\",METHOD[\"Lambert Conic Conformal (2SP)\",ID[\"EPSG\",9802]],PARAMETER[\"Latitude of false origin\",38.5,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8821]],PARAMETER[\"Longitude of false origin\",262.5,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8822]],PARAMETER[\"Latitude of 1st standard parallel\",38.5,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8823]],PARAMETER[\"Latitude of 2nd standard parallel\",38.5,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8824]],PARAMETER[\"Easting at false origin\",0,LENGTHUNIT[\"metre\",1],ID[\"EPSG\",8826]],PARAMETER[\"Northing at false origin\",0,LENGTHUNIT[\"metre\",1],ID[\"EPSG

So, unfortunately this has created a geotiff that doesn't have a CRS and that doesn't display correctly (it's way too small) even when manually given the CRS in QGIS. Did I specify this wrong somehow in the attrs or is the library broken or does it not use this non-standard CRS?

I can try:

- Loading a different example. Not sure where to find one, will check the gdal zarr driver tests if possible (have googled without results though I did find an interesting [conversation](https://github.com/pydata/xarray/issues/6448)). If that doesn't work, I'll just find some standard dataset and convert it into the format I need to check.
- Maybe posting on StackOverflow? The gdal github says not to ask questions there. They have a listserv but I didn't see a web-viewable archive to it and I really don't want to actually subscribe.